In [128]:
MODULE    = 'model'
TITLE     = 'Smalltalk-ish interactive programming environment model'
ABOUT     = ''''''

In [129]:
AUTHOR    = 'Dmitry Ponyatov'
EMAIL     = 'dponyatov@gmail.com'
YEAR      = 2021
LICENSE   = 'MIT'

COPYRIGHT = f'(c) {AUTHOR} <{EMAIL}> {YEAR} All rights reserved ({LICENSE})'

## Smalltalk-ish interactive programming environment **_model_**

In [130]:
print(COPYRIGHT)

(c) Dmitry Ponyatov <dponyatov@gmail.com> 2021 All rights reserved (MIT)


The goal of this project is to make a high-level model of an interactive environment (and programming language maybe) using the [Object graph](https://en.wikipedia.org/wiki/Object_graph) model as a core software representation layer.

The Python language and its runtime semantics look great as a base for such an interactive model, as it has a lot of advantages:
* dynamic language
* Python has a syntax that forces its readability 
* so not all like it, just use the appropriate and well-configures editor
* OOP implementation with custom operators
* popular, with huge amount of modules, and easy to learn and use

This manual is written in the [JupyterWeb](https://en.wikipedia.org/wiki/Web_(programming_system)) style, so the order of text and its detailed complexity is forced by the nature of [literate programming](https://en.wikipedia.org/wiki/Literate_programming) that kills the idea: the narrative is rigidly determined by the order of the code.

## Links

## Object graph

The object graph is an *active* data structure formed by objects containing references (pointers) to other objects.

https://en.wikipedia.org/wiki/Object_graph

As references have direction, this graph can be said precisely as a **directed hypergraph** of objects.

The idea of using object (hyper)graph for general-purpose knowledge representation can be inherited from the well-known [frames]. Thus, we can take the object graph as a core data structure not only for program representation but also for any application and user data storage.

[frames] **Marvin Minsky** [A Framework for Representing Knowledge](https://web.media.mit.edu/~minsky/papers/Frames/frames.html) <br>
MIT-AI Laboratory Memo 306, June, 1974.

* As you can see next, the base `Object` and all its inherited classes mix the nature of classic lists and maps (associative array) at the same time, so the whole system can be built upon the homogenous and active data structure.
* The `Active` means all objects and subgraphs have associated execution semantics and can be executed in some or a few ways.

In [131]:
class Object:
    def __init__(self, V):
        # scalar value: object name, number/string value,..
        self.value = V
        # slots: associative array = map = env/namespace = attributes
        self.slot  = {}
        # nested: vector = stack = nested AST subtrees
        self.nest  = []

In [132]:
hello = Object('hello'); print(hello)

print(hello.value)
print(hello.slot)
print(hello.nest)

hello
{}
[]


### Text dump for Object graph

For debugging and logging purposes we need to have a way to dump any object graph in a text form. For doing this we can start from some root node, and do recursive graph traversal with tabbing each level of recursion in output.

In [133]:
class Object(Object):
    # `print` callback
    def __repr__(self): return self.dump()
    # full text tree dump
    def dump(self, cycle=[], depth=0, prefix=''):
        # head
        def pad(depth): return '\n' + '\t' * depth
        ret = pad(depth) + self.head(prefix)
        # cycle
        if not depth: cycle = [] # recursion root
        if self in cycle: return f'{ret} _/'
        else: cycle.append(self)
        # slot{}s
        for i in self.keys():
            ret += self[i].dump(cycle, depth+1, f'{i} = ')
        # nest[]ed
        for j,k in enumerate(self):
            ret += k.dump(cycle, depth+1, f'{j}: ')
        # subgraph
        return ret
    # single line `<T:V>` header
    def head(self, prefix=''):
        gid = f' @{id(self):x}'
        return f'{prefix}<{self.tag()}:{self.val()}>{gid}'
    # `<T:`
    def tag(self): return self.__class__.__name__.lower()
    # `:V>`
    def val(self): return f'{self.value}'

### Python types boxing

Some Python types should be to be boxed inside corresponding `Primitive` graph nodes:

In [134]:
class Object(Object):
    def box(self, that):
        if isinstance(that, Object): return that
        if isinstance(that, int): return Int(that)
        if isinstance(that, str): return S(that)
        raise TypeError(['box', type(that), that])

### Operators

It is handy to define some frequently-used operations over graphs with special operators.<br>
It also includes some operations used above while dumping and iterating over graphs.

In [135]:
class Object(Object):
    # `keys()`
    def keys(self): return sorted(self.slot.keys())
    # `iter()`
    def __iter__(self): return iter(self.nest)
    # `A//B ~> A.push(B)`
    def __floordiv__(self, that):
        self.nest.append(self.box(that)) ; return self
    # `A[key]`
    def __getitem__(self, key):
        if isinstance(key, str): return self.slot[key]
        if isinstance(key, int): return self.nest[key]
        raise TypeError(['__getitem__', type(key), key])
    # `A[key] = B`
    def __setitem__(self, key, that):
        assert isinstance(key, str)
        self.slot[key] = self.box(that); return self
    # `A << B ~> A[B.type] = B`
    def __lshift__(self, that):
        that = self.box(that); return self.__setitem__(that.tag(), that)
    # `A >> B ~> A[B.value] = B`
    def __rshift__(self, that):
        that = self.box(that); return self.__setitem__(that.val(), that)    

In [136]:
hello = Object('hello')
world = Object('world')
print( hello // world )


<object:hello> @7f5c700f6240
	0: <object:world> @7f5c700f6358


In [137]:
left = Object('left') ; right = Object('right')
print( hello << left >> right)


<object:hello> @7f5c700f6240
	object = <object:left> @7f5c700f6710
	right = <object:right> @7f5c700f6780
	0: <object:world> @7f5c700f6358


## GUI

In [60]:
class GUI(Object): pass

In [61]:
from IPython.display import display
import ipywidgets as widgets

In [62]:
sli = widgets.IntSlider()
sli.description = 'test'
display(sli)

IntSlider(value=0, description='test')

In [63]:
class Slider(GUI):
    def __init__(self, V):
        super().__init__(V)
        self.slider = widgets.IntSlider()
        self.slider.description = self.head()
    def show(self):
        display(self.slider)

In [64]:
Slider('test').show()

IntSlider(value=0, description='<slider:test> @7f5c701bf588')